In [1]:
from fin_statement_model.financial_statement import FinancialStatementGraph
from fin_statement_model.builder import add_metric

In [2]:
historical_periods = ["FY2020", "FY2021", "FY2022"]
forecast_periods = ["FY2023", "FY2024"]

revenue_data = {"FY2020": 1000.0, "FY2021": 1100.0, "FY2022": 1210.0}
cogs_data = {p: revenue_data[p] * 0.4 for p in historical_periods}
opex_data = {p: revenue_data[p] * 0.2 for p in historical_periods}
interest_data = {p: 50.0 for p in historical_periods}
tax_data = {p: revenue_data[p] * 0.1 for p in historical_periods}
net_income_data = {p: (revenue_data[p] - cogs_data[p] - opex_data[p] - interest_data[p] - tax_data[p])
                    for p in historical_periods}

In [3]:
#Intialize the graph
fsg = FinancialStatementGraph(periods=[*historical_periods, *forecast_periods])

##Add historical data
# Add financial statement items for each regional revenue
fsg.add_financial_statement_item("revenue_americas", revenue_data)
fsg.add_financial_statement_item("revenue_europe", revenue_data)
fsg.add_financial_statement_item("revenue_apac", revenue_data)

# Now define a calculation node for total revenue as the sum of these three
fsg.add_calculation("revenue", ["revenue_americas", "revenue_europe", "revenue_apac"], "addition")

fsg.add_financial_statement_item("cost_of_goods_sold", cogs_data)
fsg.add_financial_statement_item("operating_expenses", opex_data)
fsg.add_financial_statement_item("interest_expense", interest_data)
fsg.add_financial_statement_item("taxes", tax_data)
fsg.add_financial_statement_item("net_income", net_income_data)

# For demonstration, assume these needed raw nodes exist or add dummy data
fsg.add_financial_statement_item("shares_outstanding", {"FY2020": 1000, "FY2021": 1000, "FY2022": 1000})
fsg.add_financial_statement_item("total_assets", {"FY2020": 5000, "FY2021": 5000, "FY2022": 5500})
fsg.add_financial_statement_item("total_equity", {"FY2020": 3000, "FY2021": 3000, "FY2022": 3200})

In [4]:
##Add forecasts
# Create curved growth forecast for Americas revenue
fsg.create_forecast("revenue_americas", "curve", "FY2022", forecast_periods, [0.05, 0.06])

# Create fixed growth forecasts for Europe and APAC revenue
fsg.create_forecast("revenue_europe", "fixed", "FY2022", forecast_periods, 0.05)
fsg.create_forecast("revenue_apac", "fixed", "FY2022", forecast_periods, 0.05)

fsg.recalculate_all()

In [5]:
fsg.add_calculation("gross_profit", ["revenue", "cost_of_goods_sold"], "subtraction")
fsg.add_calculation("operating_income", ["gross_profit", "operating_expenses"], "subtraction")

In [6]:
fsg.recalculate_all()
df = fsg.to_dataframe()
df

,FY2020,FY2021,FY2022,FY2023,FY2024
revenue_americas,1000.0,1100.0,1210.0,1270.5,1346.730
revenue_europe,1000.0,1100.0,1210.0,1270.5,1334.025
revenue_apac,1000.0,1100.0,1210.0,1270.5,1334.025
revenue,3000.0,3300.0,3630.0,3811.5,4014.780
cost_of_goods_sold,400.0,440.0,484.0,484.0,484.000
operating_expenses,200.0,220.0,242.0,242.0,242.000
interest_expense,50.0,50.0,50.0,50.0,50.000
taxes,100.0,110.0,121.0,121.0,121.000
net_income,250.0,280.0,313.0,313.0,313.000
shares_outstanding,1000.0,1000.0,1000.0,1000.0,1000.000
